In [4]:
!.venv/bin/pip3 install -r requirements.txt

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [2]:
import pandas as pd
import numpy as np
import tensorflow.keras.layers
from keras.src.layers import Lambda
from matplotlib import pyplot as plt
import tensorflow as tf
import math
import ydf  # Yggdrasil Decision Forests
from sklearn.model_selection import train_test_split
from wurlitzer import sys_pipes
import keras.layers as preprocessing
from sklearn.model_selection import KFold
import tqdm as tqdm
from sklearn.preprocessing import MinMaxScaler


%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]

2025-03-28 18:57:02.576524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743202622.587200   19209 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743202622.590472   19209 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743202622.599574   19209 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743202622.599582   19209 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743202622.599583   19209 computation_placer.cc:177] computation placer alr

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth to avoid TensorFlow using all GPU memory
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        # Set TensorFlow to use only the first GPU (if multiple GPUs available)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        print(e)

Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [5]:
def train_and_eval(model, train_ds, test_ds = None):
    # Optionally, add evaluation metrics.
    model.compile(metrics=["mse"])
    rmse = 0

    with sys_pipes():
        model.fit(x=train_ds)

    if test_ds is not None:
        evaluation = model.evaluate(x=test_ds, return_dict=True)
        rmse = math.sqrt(evaluation["mse"])

    return rmse

def latlon_to_xyz(lat, lon):
    lat, lon = np.radians(lat), np.radians(lon)
    x = np.cos(lat) * np.cos(lon)
    y = np.cos(lat) * np.sin(lon)
    z = np.sin(lat)
    return x, y, z

# Example: Normalize Cartesian coordinates between 0 and 1
def normalize_xyz(x, y, z):
    # Normalizing each coordinate to the [0, 1] range
    return (x + 1) / 2, (y + 1) / 2, (z + 1) / 2

In [35]:
ROOT_DIR = "temporary"
train_df = pd.read_csv(f'{ROOT_DIR}/Train.csv')
test_df = pd.read_csv(f'{ROOT_DIR}/Test.csv')
vocab_df = pd.read_csv(f'{ROOT_DIR}/variable_descriptions.csv')
admin_df = pd.read_csv(f'{ROOT_DIR}/zaf_adminboundaries_tabulardata.csv', sep=";")

admin_df = admin_df[["ADM4_PCODE", "AREA_SQKM", "ADM2_ID"]] # ADM3_ID
admin_df["AREA_SQKM"] = admin_df["AREA_SQKM"].str.replace(",", ".").astype(float)
train_df = pd.merge(train_df, admin_df, on="ADM4_PCODE", how="left")
test_df = pd.merge(test_df, admin_df, on="ADM4_PCODE", how="left")
label_column = "target"

default_columns = ["ward", "ADM4_PCODE"]
nul_cols = ["dw_12", "dw_13", "lan_13", "pw_08", "pw_07"] # Columns with null values
cat_columns = ["ADM2_ID"] # Categorical columns
ft_columns = default_columns + cat_columns

In [71]:
from sklearn.preprocessing import OneHotEncoder

scaler = MinMaxScaler()
# categorical_encoder = OneHotEncoder(sparse_output=False)
def preprocess_df(input_df, is_train=False):
    drop_cols = []
    df = input_df.copy()
    df = df.drop(nul_cols, axis=1)

    ## Create a new feature
    df["phi"] = df["total_individuals"] / df["total_households"]
    df["id_area"] = df["total_individuals"] / df["AREA_SQKM"]
    df["hs_area"] = df["total_households"] / df["AREA_SQKM"]
    
    ## Normalize some columns
    norm_cols = ["phi", "NL", "id_area", "hs_area"]
    df[norm_cols] = scaler.fit_transform(df[norm_cols]) if is_train else scaler.transform(df[norm_cols])

    ## Encode categorical columns
    # encoded = categorical_encoder.fit_transform(df[cat_columns]) if is_train else categorical_encoder.transform(df[cat_columns])
    # encoded_df = pd.DataFrame(encoded, columns=categorical_encoder.get_feature_names_out(cat_columns))
    # df = pd.concat([df, encoded_df], axis=1)
    drop_cols = drop_cols + cat_columns
    
    ## Transform lat and lon to xyz
    df["x"], df["y"], df["z"] = zip(*df.apply(lambda x: latlon_to_xyz(x["lat"], x["lon"]), axis=1))
    df["x"], df["y"], df["z"] = zip(*df.apply(lambda x: normalize_xyz(x["x"], x["y"], x["z"]), axis=1))

    drop_cols = drop_cols + default_columns
    if len(drop_cols) > 0:
        df.drop(drop_cols, axis=1, inplace=True)
    
    return df

## Test preprocess_df
# preprocess_df(train_ds_pd, is_train=True).head()

#### Test with categorical columns

In [32]:
X_train = preprocess_df(X_train, is_train=True)
X_val = preprocess_df(X_val, is_train=False)
train_ds_pd = pd.concat([X_train, y_train], axis=1)
test_ds_pd = pd.concat([X_val, y_val], axis=1)

ValueError: Found unknown categories ['DC39', 'DC45', 'DC38', 'DC8', 'DC7', 'DC6', 'DC40', 'DC9', 'DC37'] in column 0 during transform

In [18]:
## Check if there are some ADM2_ID values in the test set that are not in the train set
train_zones = set(train_df["ADM2_ID"].unique())
test_zones = set(test_df["ADM2_ID"].unique())
missing_zones = test_zones - train_zones
if len(missing_zones) > 0:
    print(f"Missing zones: {missing_zones}")

Missing zones: {'DC10', 'DC1', 'DC12', 'DC14', 'CPT', 'DC5', 'BUF', 'DC13', 'DC15', 'NMA', 'DC3', 'DC2', 'DC4', 'DC44'}


Can't use column ADM2_ID as a feature because it is not the same in the train and test set.

In [72]:
X = train_df.drop(label_column, axis=1)
y = train_df[label_column]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train = preprocess_df(X_train, is_train=True)
X_val = preprocess_df(X_val, is_train=False)
train_ds_pd = pd.concat([X_train, y_train], axis=1)
test_ds_pd = pd.concat([X_val, y_val], axis=1)

In [73]:
## Use ydf instead of tfdf
gbt_model = ydf.GradientBoostedTreesLearner(label=label_column, task=ydf.Task.REGRESSION, num_threads=16).train(train_ds_pd)
evaluation = gbt_model.evaluate(test_ds_pd)
print(evaluation)

Train model on 2257 examples
Model trained in 0:00:00.936360
RMSE: 3.85796
num examples: 565
num examples (weighted): 565



In [43]:
bhp_gbt_templates = ydf.GradientBoostedTreesLearner.hyperparameter_templates()
bhp_gbt_templates

{'better_defaultv1': HyperparameterTemplate(name='better_default', version=1, parameters={'growing_strategy': 'BEST_FIRST_GLOBAL'}, description='A configuration that is generally better than the default parameters without being more expensive.'),
 'benchmark_rank1v1': HyperparameterTemplate(name='benchmark_rank1', version=1, parameters={'growing_strategy': 'BEST_FIRST_GLOBAL', 'categorical_algorithm': 'RANDOM', 'split_axis': 'SPARSE_OBLIQUE', 'sparse_oblique_normalization': 'MIN_MAX', 'sparse_oblique_num_projections_exponent': 1.0}, description='Top ranking hyper-parameters on our benchmark slightly modified to run in reasonable time.')}

In [75]:
gbt_model = ydf.GradientBoostedTreesLearner(label=label_column, task=ydf.Task.REGRESSION, num_threads=16, **bhp_gbt_templates["benchmark_rank1v1"]).train(train_ds_pd)
evaluation = gbt_model.evaluate(test_ds_pd)
print(evaluation)

Train model on 2257 examples
Model trained in 0:00:01.453353
RMSE: 4.08855
num examples: 565
num examples (weighted): 565



In [76]:
gbt_model = ydf.GradientBoostedTreesLearner(label=label_column, task=ydf.Task.REGRESSION, num_threads=16, **bhp_gbt_templates["better_defaultv1"])
trainer = gbt_model.train(train_ds_pd)
evaluation = trainer.evaluate(test_ds_pd)
print(evaluation)

Train model on 2257 examples
Model trained in 0:00:00.874679
RMSE: 3.93821
num examples: 565
num examples (weighted): 565



In [56]:
evaluation.rmse, evaluation.characteristics

(3.9379325648555428, None)

- YDF achieved a RMSE of 3.87619. Let's try to tune the model. The api is also simpler than the one from TensorFlow Decision Forests.
- Using predefined hyperparameters, doesn't seem to improve the model. Let's try to tune the model.

In [51]:
all_models = {
    "RF": ydf.RandomForestLearner,
    "GBM": ydf.GradientBoostedTreesLearner,
    "CT": ydf.CartLearner,
}
unused_columns = ft_columns[0]
preprocess_function = preprocess_df
tuning_logs = {mn: {} for mn in all_models}

n_fold = 10
kf = KFold(n_splits=n_fold, shuffle=False)
hps = []
rmse = []
# Find best HP
X = train_df.drop(label_column, axis=1)
y = train_df[label_column]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

for train_index, test_index in tqdm.tqdm(kf.split(X_train), desc="Tuning"):
    X_fold_train, X_fold_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_fold_train, y_fold_test = y_train.iloc[train_index], y_train.iloc[test_index]
    X_fold_train = preprocess_function(X_fold_train, is_train=True)
    X_fold_test = preprocess_function(X_fold_test, is_train=False)

    X_fold_train["target"] = y_fold_train
    X_fold_test["target"] = y_fold_test

    for mn, md in all_models.items():
        tuner = ydf.RandomSearchTuner(num_trials=50, automatic_search_space=True)
        tuned_model = md(label=label_column, task=ydf.Task.REGRESSION, num_threads=16, tuner=tuner)
        tuned_trainer = tuned_model.train(X_fold_train)
        tuned_evaluation = tuned_model.evaluate(X_fold_test)
        rmse = tuned_evaluation.rmse
        hp = tuned_model.hyperparameters

        model_log = tuning_logs.get(mn, {})
        tuning_logs[mn] = {
            "rmse": model_log.get("rmse", []) + [rmse],
            "hp": model_log.get("hp", []) + [hp],
        }


Tuning: 0it [00:00, ?it/s]

Train model on 2031 examples


Tuning: 0it [03:37, ?it/s]


KeyboardInterrupt: 

To obtain accurate HP, we must do a cross-validation to find the best HP. We can use the RMSE as the metric to evaluate the model. But, it takes a long time to run. So, we will only tune the model with the whole training set.

In [78]:
X = train_df.drop(label_column, axis=1)
y = train_df[label_column]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

preprocess_function = preprocess_df
X_train = preprocess_function(X_train, is_train=True)
X_val = preprocess_function(X_val, is_train=False)
train_ds_pd = pd.concat([X_train, y_train], axis=1)
test_ds_pd = pd.concat([X_val, y_val], axis=1)
tuning_logs = {mn: {} for mn in all_models}

for mn, md in all_models.items():
    tuner = ydf.RandomSearchTuner(num_trials=10, automatic_search_space=True)
    tuned_model = md(label=label_column, task=ydf.Task.REGRESSION, num_threads=32, tuner=tuner)
    tuned_trainer = tuned_model.train(train_ds_pd)
    tuned_evaluation = tuned_trainer.evaluate(test_ds_pd)
    rmse = tuned_evaluation.rmse
    hp = tuned_model.hyperparameters

    model_log = tuning_logs.get(mn, {})
    tuning_logs[mn] = {
        "rmse": rmse,
        "hp": hp,
    }

tuning_logs

Train model on 2257 examples
Model trained in 0:32:12.211291
Train model on 2257 examples
Model trained in 0:08:34.274344
Train model on 2257 examples
Model trained in 0:00:34.579165


{'RF': {'rmse': 3.6206951857773455,
  'hp': {'adapt_bootstrap_size_ratio_for_maximum_training_duration': False,
   'allow_na_conditions': False,
   'bootstrap_size_ratio': 1.0,
   'bootstrap_training_dataset': True,
   'categorical_algorithm': 'CART',
   'categorical_set_split_greedy_sampling': 0.1,
   'categorical_set_split_max_num_items': -1,
   'categorical_set_split_min_item_frequency': 1,
   'compute_oob_performances': True,
   'compute_oob_variable_importances': False,
   'growing_strategy': 'LOCAL',
   'honest': False,
   'honest_fixed_separation': False,
   'honest_ratio_leaf_examples': 0.5,
   'in_split_min_examples_check': True,
   'keep_non_leaf_label_distribution': True,
   'max_depth': 16,
   'max_num_nodes': None,
   'maximum_model_size_in_memory_in_bytes': -1.0,
   'maximum_training_duration_seconds': -1.0,
   'mhld_oblique_max_num_attributes': None,
   'mhld_oblique_sample_attributes': None,
   'min_examples': 5,
   'missing_value_policy': 'GLOBAL_IMPUTATION',
   'num_c

- Setting the number of trials to 10, higher values will take a long time to run. I set it to 100 and it run for only the first model for more than 5h.

In [79]:
## Print the tuning logs
for mn, md in tuning_logs.items():
    print(f"{mn}: Best RMSE: {md['rmse']}")

RF: Best RMSE: 3.6206951857773455
GBM: Best RMSE: 3.6462261807303116
CT: Best RMSE: 5.290425706159608


We can have a better model with tuning the hyperparameters.